### Importing libraries and reading the dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from IPython.display import display

pd.options.display.max_columns = None

In [67]:
df = pd.read_csv('Features Dataset.csv')
df.head()

,Match,ID,Sport,Interviewee,Opponent,Sex,Date,Rank,Rank Opp.,Int. Age,Opp. Age,Health,Psychics,Prev. Match,Confidence,Result,Final Result
0,BG_Box_AleksandarPovetkin_vs_AnthonyDjoshua_NO,0,Box,Aleksandar Povetkin,Anthony Djoshua,Male,22.09.2018,34-1,21-0,39,28,H,5,N,3,TKO 7 (12),NO
1,BG_Box_AndiRuiz_vs_AnthonyDjoshua_YES,1,Box,Andy Ruiz,Anthony Djoshua,Male,01.06.2019,32-1,22-0,29,29,H,5,N,4,TKO 7 (12),YES
2,BG_Box_AnthonyDjoshua_vs_AleksandarPovetkin_YES,2,Box,Anthony Djoshua,Aleksandar Povetkin,Male,22.09.2018,21-0,34-1,28,39,H,5,N,4,TKO 7 (12),YES
3,BG_Box_AnthonyDjoshua_vs_AndiRuiz_NO,3,Box,Anthony Djoshua,Andy Ruiz,Male,01.06.2019,22-0,32-1,29,29,H,5,N,4,TKO 7 (12),NO
4,BG_Box_AntonyDjoshua_vs_VladimirKlichko_YES,4,Box,Anthony Djoshua,Wladimir Klitschko,Male,29.04.2017,18-0,64-4,27,41,H,5,N,4,TKO 11 (12),YES


In [68]:
df.index = list(df.ID)
df.drop(['Match', 'ID', 'Result'], axis=1, inplace=True)

In [69]:
df.head()

,Sport,Interviewee,Opponent,Sex,Date,Rank,Rank Opp.,Int. Age,Opp. Age,Health,Psychics,Prev. Match,Confidence,Final Result
0,Box,Aleksandar Povetkin,Anthony Djoshua,Male,22.09.2018,34-1,21-0,39,28,H,5,N,3,NO
1,Box,Andy Ruiz,Anthony Djoshua,Male,01.06.2019,32-1,22-0,29,29,H,5,N,4,YES
2,Box,Anthony Djoshua,Aleksandar Povetkin,Male,22.09.2018,21-0,34-1,28,39,H,5,N,4,YES
3,Box,Anthony Djoshua,Andy Ruiz,Male,01.06.2019,22-0,32-1,29,29,H,5,N,4,NO
4,Box,Anthony Djoshua,Wladimir Klitschko,Male,29.04.2017,18-0,64-4,27,41,H,5,N,4,YES


### Transforming and removing attributes

In [70]:
df["Date"] = pd.to_datetime(df["Date"] )
df.head()

,Sport,Interviewee,Opponent,Sex,Date,Rank,Rank Opp.,Int. Age,Opp. Age,Health,Psychics,Prev. Match,Confidence,Final Result
0,Box,Aleksandar Povetkin,Anthony Djoshua,Male,2018-09-22,34-1,21-0,39,28,H,5,N,3,NO
1,Box,Andy Ruiz,Anthony Djoshua,Male,2019-01-06,32-1,22-0,29,29,H,5,N,4,YES
2,Box,Anthony Djoshua,Aleksandar Povetkin,Male,2018-09-22,21-0,34-1,28,39,H,5,N,4,YES
3,Box,Anthony Djoshua,Andy Ruiz,Male,2019-01-06,22-0,32-1,29,29,H,5,N,4,NO
4,Box,Anthony Djoshua,Wladimir Klitschko,Male,2017-04-29,18-0,64-4,27,41,H,5,N,4,YES


In [71]:
column_date = df['Date']

df_date = pd.DataFrame({"year": column_date.dt.year,
                        "dayofyear": column_date.dt.dayofyear,
                        "week": column_date.dt.week,
                        "dayofweek": column_date.dt.dayofweek,
                       })

In [72]:
df.drop('Date', axis=1, inplace=True)

In [73]:
df_date.head()

,year,dayofyear,week,dayofweek
0,2018,265,38,5
1,2019,6,1,6
2,2018,265,38,5
3,2019,6,1,6
4,2017,119,17,5


In [74]:
df = pd.concat([df, df_date], axis=1)

In [75]:
def rank_transform(v):
    v = v.replace('-', ' ')
    split = v.split(' ')
    wins = float(split[0])
    loses = float(split[1])
    loses = loses if loses > 0 else 0.1
    rank = wins / loses
    return rank

In [76]:
df['is_male'] = df['Sex'].map(lambda s: 1 if s == 'Male' else 0)

In [77]:
df.drop('Sex', axis=1, inplace=True)

In [78]:
df['Rank_T'] = 0

In [79]:
for index, row in df.iterrows():
    if row['Sport'] != 'Tennis':
        df.loc[(df.index==index), 'Rank_T'] = rank_transform(row['Rank'])
    else:
        df.loc[(df.index==index), 'Rank_T'] = float(row['Rank'])

In [80]:
df['Rank'] = df['Rank_T']

In [81]:
df['Rank_T'] = 0

In [82]:
df.head()

,Sport,Interviewee,Opponent,Rank,Rank Opp.,Int. Age,Opp. Age,Health,Psychics,Prev. Match,Confidence,Final Result,year,dayofyear,week,dayofweek,is_male,Rank_T
0,Box,Aleksandar Povetkin,Anthony Djoshua,34.0,21-0,39,28,H,5,N,3,NO,2018,265,38,5,1,0
1,Box,Andy Ruiz,Anthony Djoshua,32.0,22-0,29,29,H,5,N,4,YES,2019,6,1,6,1,0
2,Box,Anthony Djoshua,Aleksandar Povetkin,210.0,34-1,28,39,H,5,N,4,YES,2018,265,38,5,1,0
3,Box,Anthony Djoshua,Andy Ruiz,220.0,32-1,29,29,H,5,N,4,NO,2019,6,1,6,1,0
4,Box,Anthony Djoshua,Wladimir Klitschko,180.0,64-4,27,41,H,5,N,4,YES,2017,119,17,5,1,0


In [83]:
for index, row in df.iterrows():
    if row['Sport'] != 'Tennis':
        df.loc[(df.index==index), 'Rank_T'] = rank_transform(row['Rank Opp.'])
    else:
        df.loc[(df.index==index), 'Rank_T'] = float(row['Rank Opp.'])

In [84]:
df['Rank Opp.'] = df['Rank_T']

In [85]:
df.drop('Rank_T', axis=1, inplace=True)

In [86]:
df['Final Result'] = df['Final Result'].apply(lambda x: 1 if x=='YES' else 0)

In [87]:
df.head()

,Sport,Interviewee,Opponent,Rank,Rank Opp.,Int. Age,Opp. Age,Health,Psychics,Prev. Match,Confidence,Final Result,year,dayofyear,week,dayofweek,is_male
0,Box,Aleksandar Povetkin,Anthony Djoshua,34.0,210.0,39,28,H,5,N,3,0,2018,265,38,5,1
1,Box,Andy Ruiz,Anthony Djoshua,32.0,220.0,29,29,H,5,N,4,1,2019,6,1,6,1
2,Box,Anthony Djoshua,Aleksandar Povetkin,210.0,34.0,28,39,H,5,N,4,1,2018,265,38,5,1
3,Box,Anthony Djoshua,Andy Ruiz,220.0,32.0,29,29,H,5,N,4,0,2019,6,1,6,1
4,Box,Anthony Djoshua,Wladimir Klitschko,180.0,16.0,27,41,H,5,N,4,1,2017,119,17,5,1


In [88]:
df['is_int_prime_age'] = df['Int. Age'].apply(lambda x: 1 if 23 <= x <= 28 else 0)

In [89]:
df['is_opp_prime_age'] = df['Opp. Age'].apply(lambda x: 1 if 23 <= x <= 28 else 0)

In [90]:
df['age_gap'] = df['Int. Age'] - df['Opp. Age']

In [91]:
df.head()

,Sport,Interviewee,Opponent,Rank,Rank Opp.,Int. Age,Opp. Age,Health,Psychics,Prev. Match,Confidence,Final Result,year,dayofyear,week,dayofweek,is_male,is_int_prime_age,is_opp_prime_age,age_gap
0,Box,Aleksandar Povetkin,Anthony Djoshua,34.0,210.0,39,28,H,5,N,3,0,2018,265,38,5,1,0,1,11
1,Box,Andy Ruiz,Anthony Djoshua,32.0,220.0,29,29,H,5,N,4,1,2019,6,1,6,1,0,0,0
2,Box,Anthony Djoshua,Aleksandar Povetkin,210.0,34.0,28,39,H,5,N,4,1,2018,265,38,5,1,1,0,-11
3,Box,Anthony Djoshua,Andy Ruiz,220.0,32.0,29,29,H,5,N,4,0,2019,6,1,6,1,0,0,0
4,Box,Anthony Djoshua,Wladimir Klitschko,180.0,16.0,27,41,H,5,N,4,1,2017,119,17,5,1,1,0,-14


### Normalization

In [92]:
scaled_cols = df[['Rank','Rank Opp.', 'Int. Age', 'Opp. Age', 'age_gap']]
min_max_scaler = preprocessing.MinMaxScaler()
scaled_cols = min_max_scaler.fit_transform(scaled_cols)
df[['Rank','Rank Opp.', 'Int. Age', 'Opp. Age', 'age_gap']] = scaled_cols

In [93]:
df.head()

,Sport,Interviewee,Opponent,Rank,Rank Opp.,Int. Age,Opp. Age,Health,Psychics,Prev. Match,Confidence,Final Result,year,dayofyear,week,dayofweek,is_male,is_int_prime_age,is_opp_prime_age,age_gap
0,Box,Aleksandar Povetkin,Anthony Djoshua,0.125461,0.774908,0.948718,0.409091,H,5,N,3,0,2018,265,38,5,1,0,1,0.921053
1,Box,Andy Ruiz,Anthony Djoshua,0.118081,0.811808,0.692308,0.454545,H,5,N,4,1,2019,6,1,6,1,0,0,0.631579
2,Box,Anthony Djoshua,Aleksandar Povetkin,0.774908,0.125461,0.666667,0.909091,H,5,N,4,1,2018,265,38,5,1,1,0,0.342105
3,Box,Anthony Djoshua,Andy Ruiz,0.811808,0.118081,0.692308,0.454545,H,5,N,4,0,2019,6,1,6,1,0,0,0.631579
4,Box,Anthony Djoshua,Wladimir Klitschko,0.664207,0.059041,0.641026,1.000000,H,5,N,4,1,2017,119,17,5,1,1,0,0.263158


### Converts columns to categorical dtype and create dummy columns

In [95]:
df[['Interviewee', 'Opponent', 'Health', 'Psychics', 'Prev. Match', 'Confidence',
   'year', 'dayofyear', 'week', 'dayofweek', 'is_male',
   'is_int_prime_age', 'is_opp_prime_age', 'age_gap']] = df[['Interviewee', 'Opponent', 'Health', 'Psychics', 'Prev. Match', 'Confidence',
                                                           'year', 'dayofyear', 'week', 'dayofweek', 'is_male',
                                                           'is_int_prime_age', 'is_opp_prime_age', 'age_gap']].astype('category')

In [96]:
df = pd.get_dummies(df, drop_first=True)
print(df.shape)

(50, 176)


In [98]:
df.iloc[-5:]

,Rank,Rank Opp.,Int. Age,Opp. Age,Final Result,Sport_MMA,Sport_Tennis,Interviewee_Andy Mury,Interviewee_Andy Ruiz,Interviewee_Anita Doganova,Interviewee_Anthony Djoshua,Interviewee_Blagoi Ivanov,Interviewee_Bogdan Dinu,Interviewee_Cvetana Pironkova,Interviewee_Daniel Zlatkov,Interviewee_David Gofen,Interviewee_Dereck Chisora,Interviewee_Deyan Topalski,Interviewee_Dominic Thiem,Interviewee_Fabio Fonini,Interviewee_Grigor Dimitrov,Interviewee_Hughie Fury,Interviewee_Kevin Johnson,Interviewee_Kubrat Pulev,Interviewee_Maria Sharapova,Interviewee_Mario Cilic,Interviewee_Nikolas Basilashvili,Interviewee_Novak Djokovic,Interviewee_Rafael Nadal,Interviewee_Roger Federer,Interviewee_Stan Wawrinka,Interviewee_Stefanos Cicipas,Interviewee_Svetlozar Savov,Interviewee_Tervel Pulev,Interviewee_Tommy Robredo,Interviewee_Tsvetozar Iliev,Interviewee_Tyson Fury,Interviewee_Wladimir Klitschko,Opponent_Aleksandra Toncheva,Opponent_Andy Ruiz,Opponent_Anthony Djoshua,Opponent_Atanas Mihaylov,Opponent_Ben Rothwell,Opponent_Bogdan Dinu,Opponent_Caroline Wozniacki,Opponent_Cvetana Pironkova,Opponent_David Gofen,Opponent_Dereck Chisora,Opponent_Dominic Thiem,Opponent_Francesco Pianeta,Opponent_Gael Montfis,Opponent_Grigor Dimitrov,Opponent_Grigor Saruhanian,Opponent_Hughie Fury,Opponent_Jerzy Janowicz,Opponent_Kevin Johnson,Opponent_Kubrat Pulev,Opponent_Leonardo Bruzzese,Opponent_Marcos Bagdatis,Opponent_Maurice Harris,Opponent_Muslim Salihov,Opponent_Nikolas Basilashvili,Opponent_Piotr Strus,Opponent_Rafael Nadal,Opponent_Roger Federer,Opponent_Stan Wawrinka,Opponent_Tommy Robredo,Opponent_Tony Thompson,Opponent_Viktor Troicky,Opponent_Wladimir Klitschko,Health_H,Health_S,Psychics_3,Psychics_4,Psychics_5,Prev. Match_N,Prev. Match_W,Prev. Match_Y,Confidence_2,Confidence_3,Confidence_4,Confidence_5,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,dayofyear_13,dayofyear_25,dayofyear_27,dayofyear_39,dayofyear_42,dayofyear_64,dayofyear_82,dayofyear_85,dayofyear_108,dayofyear_118,dayofyear_119,dayofyear_132,dayofyear_133,dayofyear_149,dayofyear_151,dayofyear_180,dayofyear_187,dayofyear_218,dayofyear_230,dayofyear_236,dayofyear_245,dayofyear_246,dayofyear_247,dayofyear_265,dayofyear_275,dayofyear_278,dayofyear_280,dayofyear_300,dayofyear_306,dayofyear_319,dayofyear_320,dayofyear_332,week_2,week_4,week_6,week_7,week_10,week_12,week_16,week_17,week_20,week_22,week_26,week_27,week_32,week_33,week_34,week_35,week_36,week_38,week_40,week_43,week_44,week_46,week_48,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,is_male_1,is_int_prime_age_1,is_opp_prime_age_1,age_gap_0.2631578947368421,age_gap_0.2894736842105263,age_gap_0.3157894736842105,age_gap_0.3421052631578947,age_gap_0.39473684210526316,age_gap_0.47368421052631576,age_gap_0.5,age_gap_0.5263157894736842,age_gap_0.5526315789473684,age_gap_0.5789473684210527,age_gap_0.6052631578947368,age_gap_0.631578947368421,age_gap_0.6578947368421052,age_gap_0.6842105263157894,age_gap_0.7105263157894737,age_gap_0.7368421052631579,age_gap_0.763157894736842,age_gap_0.7894736842105263,age_gap_0.8684210526315789,age_gap_0.9210526315789473,age_gap_0.9736842105263157,age_gap_1.0
45,0.011070,0.014760,0.923077,0.272727,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
46,0.018450,0.033210,0.846154,0.318182,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
47,0.103321,0.191882,0.820513,0.409091,1,0,1,0

In [100]:
df = df.rename(columns={'Final Result': 'Class'})

### Write resulting df to csv file

In [101]:
df.to_csv('transformed_structured_data.csv')